In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import concurrent.futures
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras

from keras import Sequential
from keras import regularizers
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,Dropout,BatchNormalization
from keras.callbacks import Callback,EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix
)
import glob
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection

In [ ]:
def combine_data(path):
    dirs = os.listdir(path)
    combine_df = []
    for filepath in dirs:
        source = filepath.split('.')[0]
        if filepath.endswith('.csv'):
            X = pd.read_csv(path + filepath, parse_dates=['timestamp'], index_col='timestamp')
            X['source'] = source
            combine_df.append(X)

    return combine_df

In [ ]:
combine_df = combine_data('/content/drive/MyDrive/Motion_Activity/data/condition/')

In [ ]:
conditions = []
for condition in combine_df:
    condition_df = pd.DataFrame(columns=['mean_activity', 'std_activity', 'zero_activity_proportion', 'source'])
    condition_df['mean_activity'] = condition.activity.resample('H').mean()
    condition_df['std_activity'] = condition.activity.resample('H').std()
    condition_df['zero_activity_proportion'] = [data[1].tolist().count(0) for data in condition.activity.resample('H')]
    condition_df['source'] = condition.source
    conditions.append(condition_df)

In [ ]:
combine_df = combine_data('/content/drive/MyDrive/Motion_Activity/data/control/')

In [ ]:
controls = []
for control in combine_df:
    control_df = pd.DataFrame(columns=['mean_activity', 'std_activity', 'zero_activity_proportion', 'source'])
    control_df['mean_activity'] = control.activity.resample('H').mean()
    control_df['std_activity'] = control.activity.resample('H').std()
    control_df['zero_activity_proportion'] = [data[1].tolist().count(0) for data in control.activity.resample('H')]
    control_df['source'] = control.source
    controls.append(control_df)

In [ ]:
def nextday(dates):
    for date in dates:
        yield date
def zero_count(series):
    return list(series).count(0)
def extractfeatures(X, date):
    mask = X['date'] == date
    d = {
        'mean_log_activity': X[mask]['log_activity'].mean(),
        'std_log_activity': X[mask]['log_activity'].std(),
        'min_log_activity': X[mask]['log_activity'].min(),
        'max_log_activity': X[mask]['log_activity'].max(),
        'zero_proportion_activity': zero_count(X[mask]['log_activity'])
    }
    return d

In [ ]:
class ExtractData(BaseEstimator, TransformerMixin):

    def __init__(self, path):
        self.path = path
        self.X = []

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        dirs = os.listdir(self.path)

        for filepath in sorted(dirs, key=lambda x: x.split('_')[0]):
            condition = filepath.split('.')[0]
            if filepath.endswith('.csv'):
                X = pd.read_csv(self.path + filepath)
                X['log_activity'] = np.log(X['activity'] + 1)
                dates = X.date.unique()

                for date in nextday(dates):
                    d = extractfeatures(X, date)
                    d['source'] = condition
                    self.X.append(d)


        return pd.DataFrame(self.X)

In [ ]:
e = ExtractData(path='/content/drive/MyDrive/Motion_Activity/data/condition/')
conditions = e.fit_transform(X=None, y=None)
conditions['state'] = 1

In [ ]:
e = ExtractData(path='/content/drive/MyDrive/Motion_Activity/data/control/')
controls = e.fit_transform(X=None, y=None)
controls['state'] = 0

In [ ]:
full_df = controls.append(conditions, ignore_index=True)
full_df.head()

,mean_log_activity,std_log_activity,min_log_activity,max_log_activity,zero_proportion_activity,source,state
0,5.232499,1.494106,0.0,7.976595,24,control_28,0
1,4.103117,2.658716,0.0,8.657129,359,control_28,0
2,4.057168,2.617790,0.0,8.007367,366,control_28,0
3,4.094460,2.587575,0.0,7.852050,350,control_28,0
4,4.118694,2.542314,0.0,8.100768,344,control_28,0


In [ ]:
def custom_train_test_split(train_set, test_set):
    X_train = train_set.drop('label', axis=1)
    y_train = train_set.label
    X_test = test_set.drop('label', axis=1)
    y_test = test_set.label

    return X_train, X_test, y_train, y_test

In [ ]:
class CustomClassifierCV(BaseEstimator, TransformerMixin):

    def __init__(self, base_clf):
        self.base_clf = base_clf

    def fit(self, X, y=None):
        X['label'] = y
        participants = X.source.unique()
        folds = []

        predictions = []
        prob=[]
        actuals = []

        for p in participants:
            folds.append(X[X['source'] == p])

        for i in range(len(folds)):
            test_set = folds[i]
            train_fold = [elem for idx , elem in enumerate(folds) if idx != i]

            train_set = pd.concat(train_fold)
            X_train, X_test, y_train, y_test = custom_train_test_split(train_set.drop(['source'], axis=1),test_set.drop(['source'], axis=1))

            self.base_clf.fit(X_train, y_train)
            prob.append(self.base_clf.predict_proba(X_test)[:, 1])
            predictions.append(self.predict(X_test))
            actuals.append(test_set.label.iloc[0])

            # self.score(predictions, actuals)
            self.probabilities(prob)

    def predict(self, X):
        predictions = self.base_clf.predict(X)
        ones = predictions.tolist().count(1)
        zeroes = predictions.tolist().count(0)

        return 1 if ones > zeroes else 0
    def probabilities(self,prob):
        for i in prob:
          print(i)


    def score(self, predictions, actuals):
        print(classification_report(predictions, actuals))

In [ ]:
X = full_df.drop(['state'], axis=1)
y = full_df.state
X_train, X_test, y_train, y_test = custom_train_test_split(train_set.drop(['source'], axis=1),test_set.drop(['source'], axis=1))
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train,y_train)
custom_clfCV = CustomClassifierCV(forest)
# forest.predict_proba(X_test)[:, 1]
custom_clfCV.fit(X, y)
# custom_clfCV.probabilities()

array([0.47, 0.77, 0.33, 0.31, 0.34, 0.6 , 0.48, 0.67, 0.28, 0.45, 0.5 ,
       0.62, 0.49, 0.18, 0.64, 0.42])

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
custom_clfCV = CustomClassifierCV(knn)
custom_clfCV.fit(X, y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg     

In [ ]:
xgb_classifier = xgboost.XGBClassifier()
custom_clfCV = CustomClassifierCV(xgb_classifier)
custom_clfCV.fit(X, y)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg     

In [ ]:
X['label'] = y
participants = X.source.unique()
folds = []

predictions = []
actuals = []

for p in participants:
    folds.append(X[X['source'] == p])

for i in range(len(folds)):
    test_set = folds[i]

train_fold = [elem for idx , elem in enumerate(folds) if idx != i]
train_set = pd.concat(train_fold)

X_train, X_test, y_train, y_test = custom_train_test_split(train_set.drop(['source'], axis=1),test_set.drop(['source'], axis=1))

estimators = []
model1 = forest; estimators.append(("forest", model1))
model2 = knn; estimators.append(("knn", model2))
model3 = xgb_classifier; estimators.append(("xgb_classifier", model3))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X_train, y_train)
ml_model=results.mean()
print(ml_model)

0.6667099311701081


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

X_train = np.reshape(X_train.drop('source', axis=1).values, (X_train.shape[0], 1, X_train.shape[1]-1))
X_test = np.reshape(X_test.drop('source', axis=1).values, (X_test.shape[0], 1, X_test.shape[1]-1))

X_train = X_train.astype(float)
X_test = X_test.astype(float)

model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,epochs=15)

model1 = model.predict(X_test)

Epoch 1/15
29/29 [==============================] - 3s 21ms/step - loss: 0.6661 - accuracy: 0.6230 - val_loss: 0.6199 - val_accuracy: 0.6856
Epoch 2/15
29/29 [==============================] - 0s 4ms/step - loss: 0.6456 - accuracy: 0.6437 - val_loss: 0.6082 - val_accuracy: 0.7118
Epoch 3/15
29/29 [==============================] - 0s 4ms/step - loss: 0.6378 - accuracy: 0.6612 - val_loss: 0.6117 - val_accuracy: 0.7118
Epoch 4/15
29/29 [==============================] - 0s 4ms/step - loss: 0.6369 - accuracy: 0.6601 - val_loss: 0.5972 - val_accuracy: 0.7118
Epoch 5/15
29/29 [==============================] - 0s 4ms/step - loss: 0.6294 - accuracy: 0.6656 - val_loss: 0.5889 - val_accuracy: 0.7118
Epoch 6/15
29/29 [==============================] - 0s 4ms/step - loss: 0.6229 - accuracy: 0.6667 - val_loss: 0.5827 - val_accuracy: 0.7118
Epoch 7/15
29/29 [==============================] - 0s 5ms/step - loss: 0.6195 - accuracy: 0.6678 - val_loss: 0.5797 - val_accuracy: 0.7162
Epoch 8/15
29/29 [=

In [ ]:
import IPython.display as ipd

audio_fpath = '/content/drive/MyDrive/audio_files/'
audio_clips = os.listdir(audio_fpath)
print("No. of .wav files in audio folder = ",len(audio_clips))

No. of .wav files in audio folder =  188


In [ ]:
def plot_spectrogram_for_audio(audio_fpath, output_dir, sr=11025):
    x, sr = librosa.load(audio_fpath, sr=sr)

    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X)0)

    filename = os.path.splitext(os.path.basename(audio_fpath))[0]
    output_file_path = os.path.join(output_dir, f'{filename}_spectrogram.png')

    plt.figure(figsize=(14, 5))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()
    plt.savefig(output_file_path)
    plt.close()

if __name__ == "__main__":
    audio_directory = '/content/drive/MyDrive/audio_files/'
    output_directory = '/content/drive/MyDrive/spectrogram_output/'
    sample_rate = 11025

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    audio_files = [file for file in os.listdir(audio_directory) if file.endswith(".wav")]

    for audio_file in audio_files:
        audio_fpath = os.path.join(audio_directory, audio_file)
        print(f"Plotting and saving spectrogram for: {audio_fpath}")
        plot_spectrogram_for_audio(audio_fpath, output_directory, sr=sample_rate)

In [ ]:
train_csv1=pd.read_csv('/content/drive/MyDrive/train_split.csv').set_index('Participant_ID')
test_csv=pd.read_csv('/content/drive/MyDrive/full_test_split.csv').set_index('Participant_ID')
dev_csv=pd.read_csv('/content/drive/MyDrive/dev_split_Depression_AVEC2017.csv').set_index('Participant_ID')

train_csv=pd.concat([train_csv1,dev_csv])

In [ ]:
total=train_csv.shape[0]
input_path="/content/drive/MyDrive/spectrogram_output/"
X_train = np.empty(shape=(total, 500, 1400, 3))
y_train = np.empty(shape=(total),dtype='int')
idx=0

for dir in os.listdir(input_path):
  for i in train_csv.index:
    if int(dir[:3]) == int(i):
      X_train[idx] = np.expand_dims(cv2.imread(input_path + dir, 0), axis=2)
      y_train[idx] = int(train_csv['PHQ8_Binary'][i])
      idx += 1



total=test_csv.shape[0]
X_test = np.empty(shape=(total, 500, 1400, 3))
y_test = np.empty(shape=(total),dtype='int')
idx=0

for dir in os.listdir(input_path):
  for i in test_csv.index:
    if int(dir[:3]) == int(i):
      X_test[idx] = np.expand_dims(cv2.imread(input_path + dir, 0), axis=2)
      y_test[idx] = int(test_csv['PHQ_Binary'][i])
      idx += 1

In [ ]:
model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(500, 1400, 3)))
model.add(MaxPooling2D((4, 4), strides=4))
model.add(Conv2D(32, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D((1, 3), strides=(1,3)))
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.6))
model.add(Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
            loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
X_train=X_train/255
X_test=X_test/255

In [ ]:
callbacks=EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=12,
    verbose=1,
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
# datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
# train_generator = datagen.flow_from_directory(
#         X_train,
#         batch_size=10,
#         target_size=(512,512),
#         class_mode='binary')
# val_generator = datagen.flow_from_directory(
#         X_test,
#         color_mode='rgb',
#         target_size=(512,512),
#         class_mode='binary')

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=10,epochs=20,callbacks=callbacks)

Epoch 1/20
15/15 [==============================] - 135s 9s/step - loss: 9.0963 - accuracy: 0.5845 - val_loss: 3.9824 - val_accuracy: 0.7021
Epoch 2/20
15/15 [==============================] - 129s 9s/step - loss: 3.6424 - accuracy: 0.6549 - val_loss: 3.0731 - val_accuracy: 0.7021
Epoch 3/20
15/15 [==============================] - 138s 9s/step - loss: 2.7229 - accuracy: 0.6620 - val_loss: 2.2809 - val_accuracy: 0.7021
Epoch 4/20
15/15 [==============================] - 142s 10s/step - loss: 2.1267 - accuracy: 0.6549 - val_loss: 1.8295 - val_accuracy: 0.7021
Epoch 5/20
15/15 [==============================] - 138s 9s/step - loss: 1.6990 - accuracy: 0.6901 - val_loss: 1.5211 - val_accuracy: 0.7021
Epoch 6/20
15/15 [==============================] - 140s 10s/step - loss: 1.4307 - accuracy: 0.7042 - val_loss: 1.3333 - val_accuracy: 0.7021
Epoch 7/20
15/15 [==============================] - 136s 9s/step - loss: 1.2850 - accuracy: 0.6972 - val_loss: 1.1954 - val_accuracy: 0.7021
Epoch 8/20


In [ ]:
model2=model.predict(X_test)
for i in model2:
  r2=i[1]
  print(r2)


In [ ]:
input_path="/content/drive/MyDrive/CK+48/"

total=0
for dir_ in os.listdir(input_path):
    count=0
    for f in os.listdir(input_path+dir_+"/"):
        count+=1
    total+=count

total

750

In [ ]:
TOP_EMOTIONS = ["anger", "fear", "happy", "sadness", "surprise"]
# dep_emotion_score=[-0.1,-0.45,0.5,0.2,0.1]

In [ ]:
from keras.src.utils import np_utils
img_arr = np.empty(shape=(total, 48, 48, 1))
img_label=np.empty(shape=(total))
idx=0
label_to_text={}
label=0
for dir_ in os.listdir(input_path):
    if dir_ in TOP_EMOTIONS:
      for f in os.listdir(input_path + dir_ + "/"):
            img_arr[idx] = np.expand_dims(cv2.imread(input_path + dir_ + "/" + f, 0), axis=2)
            img_label[idx] = label
            idx += 1
      label_to_text[label] = dir_
      label += 1
img_label = np_utils.to_categorical(img_label)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(img_arr,img_label,stratify=img_label,shuffle=True)

In [ ]:
X_train=X_train/255
X_test=X_test/255

In [ ]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(3,3),activation='elu',padding='same',input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=(3,3),activation='elu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.3))
model.add(Conv2D(128,kernel_size=(3,3),activation='elu',padding='same',kernel_regularizer=regularizers.l2(l=0.01)))
model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size=(3,3),activation='elu',padding='same',kernel_regularizer=regularizers.l2(l=0.01)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.3))
model.add(Conv2D(256,kernel_size=(3,3),activation='elu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size=(3,3),activation='elu',padding='same'))
model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dropout(0.2))
model.add(Dense(128,activation='elu'))
model.add(Dense(len(label_to_text),activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
callbacks=EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=12,
    verbose=1,
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
# train_datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.15,
#     height_shift_range=0.15,
#     shear_range=0.15,
#     zoom_range=0.15,
#     horizontal_flip=True,
# )
# train_datagen.fit(X_train)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,epochs=20,callbacks=callbacks)

Epoch 1/20
18/18 [==============================] - 41s 2s/step - loss: 5.1270 - accuracy: 0.5552 - val_loss: 3.5560 - val_accuracy: 0.6596
Epoch 2/20
18/18 [==============================] - 41s 2s/step - loss: 2.5410 - accuracy: 0.8310 - val_loss: 2.9521 - val_accuracy: 0.6968
Epoch 3/20
18/18 [==============================] - 37s 2s/step - loss: 2.2589 - accuracy: 0.9181 - val_loss: 2.3821 - val_accuracy: 0.8617
Epoch 4/20
18/18 [==============================] - 41s 2s/step - loss: 2.0663 - accuracy: 0.9395 - val_loss: 2.0046 - val_accuracy: 0.9415
Epoch 5/20
18/18 [==============================] - 37s 2s/step - loss: 1.8901 - accuracy: 0.9537 - val_loss: 2.0436 - val_accuracy: 0.8989
Epoch 6/20
18/18 [==============================] - 39s 2s/step - loss: 1.7812 - accuracy: 0.9520 - val_loss: 1.7018 - val_accuracy: 0.9681
Epoch 7/20
18/18 [==============================] - 39s 2s/step - loss: 1.5947 - accuracy: 0.9662 - val_loss: 1.6863 - val_accuracy: 0.9309
Epoch 8/20
18/18 [==

In [ ]:
y # def predict(arr):
#   yhat_test=0
#   l=[]
#   for i in model.predict(arr):
#     for j in range(5):
#       yhat_test+=i[j]*dep_emotion_score[j]
#       if yhat_test>0.5:
#         l.append(1)
#       else:
#         l.append(0)
#   return l

model3=model.predict(X_test)

# TOP_EMOTIONS = ["anger", "fear", "happy", "sadness", "surprise"]

# model3

for i in model3:
  neg_rate=((i[0]+i[1]+i[3])/3)
  # pos_rate=round(i[2]*100)
  print(neg_rate)
  # print(he)


6/6 [==============================] - 5s 787ms/step
0.09981473286946614
0.10530083378156026
0.09589712818463643
0.1052472988764445
0.09346073865890503
0.10490005215009053
0.08663801352183025
0.09804246822992961
0.09361215432484944
0.08659516771634419
0.10707132021586101
0.10540289680163066
0.09605816006660461
0.10154672463734944
0.1041897435983022
0.09674935539563496
0.10531455278396606
0.09526009360949199
0.09038035074869792
0.10146841406822205
0.09333897630373637
0.10595317681630452
0.10940283536911011
0.09639513492584229
0.09942059715588887
0.10753427942593892
0.11243779460589091
0.10067494710286458
0.10774417718251546
0.09490158160527547
0.10937122503916423
0.09848759571711223
0.09985878070195515
0.10274955630302429
0.10390033324559529
0.09661078453063965
0.09735108415285747
0.10580540696779887
0.09052111705144246
0.08956097563107808
0.09487632910410564
0.09480157494544983
0.09773480892181396
0.10761246085166931
0.09845404823621114
0.09973196188608806
0.09356066584587097
0.1011858

In [ ]:
model3=model.predict(X_test)

# TOP_EMOTIONS = ["anger", "fear", "happy", "sadness", "surprise"]

# model3

for i in model3:
  neg_rate=((i[0]+i[1]+i[3])/3)
  # pos_rate=round(i[2]*100)
  print(neg_rate)
  # print(he)

In [ ]:
# model3= np.round(model3).astype(int)
# model2= np.round(model2).astype(int)
# model1= np.round(model1).astype(int)
# # ml_model=np.round(ml_model).astype(int)
# model1 = model2.reshape((-1,))  # Reshape to a 1D array
# model2 = model2.reshape((-1,))
# model3 = model3.reshape((-1,))

# ensemble_predictions = np.round( model1 + model2 + model3) / 3

# # Make final prediction based on majority voting
# final_predictions = np.where(ensemble_predictions >= 0.5, 1, 0)

# # Calculate ensemble accuracy
# ensemble_accuracy = accuracy_score(y_test, final_predictions)
# print("Ensemble Accuracy:", ensemble_accuracy)

In [ ]:
if (neg_rate <= 30.0):
  if (neg_rate <= 5.0):
    print("just chill and relax ")
  else:
    print("\nDepression Scale Detected : ", round(neg_rate) / 10,"\n")
    print("***   MILD DEPRESSION  ***")

elif (neg_rate > 30.0 and neg_rate <= 60.0):
  print("\nDepression Scale Detected : ", round(neg_rate) / 10,"\n")
  print("***   MODERATE DEPRESSION  ***")

elif (neg_rate > 60.0 and neg_rate <= 100.0):
  print("\nDepression Scale Detected : ", round(neg_rate) / 10,"\n")
  print("***   SEVERE DEPRESSION  ***")



In [ ]:
def ensemble(r1,r2,r3):
  avg=(r1+r2+r3)/3
  if avg<0.5:
    return 0
  return 1

